# Traveling Salesman Problem

## Objective and Prerequisites

In this example, you’ll learn how to tackle one of the most famous combinatorial optimization problems in existence: the Traveling Salesman Problem (TSP). The goal of the TSP – to find the shortest possible route that visits each city once and returns to the original city – is simple, but solving the problem is a complex and challenging endeavor. We’ll show you how to do it!

This modeling example is at the advanced level, where we assume that you know Python and the Gurobi Python API and that you have advanced knowledge of building mathematical optimization models. Typically, the objective function and/or constraints of these examples are complex or require advanced features of the Gurobi Python API.

**Gurobi Offical TSP demo** <br />
You can find the offical example by clicking [here](https://gurobi.github.io/modeling-examples/traveling_salesman/).

## Motivation

The Traveling Salesman Problem (TSP) is one of the most famous combinatorial optimization problems. This problem is very easy to explain, but very complicated to solve – even for instances with a small number of cities. More detailed information on the TSP can be found in the book The Traveling Salesman Problem: A Computational Study [1], or at the TSP home page [2]. If you are interested in the history and mathematical background of the TSP, we recommend that you watch the video by William Cook [3].

The origin of the traveling salesman problem is not very clear; it is mentioned in an 1832 manual for traveling salesman, which included example tours of 45 German cities but was not formulated as a mathematical problem. However, in the 1800s, mathematicians William Rowan Hamilton and Thomas Kirkman devised mathematical formulations of the problem.

It seems that the general form of the Traveling Salesman Problem was first studied by Karl Menger in Vienna and Harvard in the 1930s.

The problem became more and more popular in the 1950s and 1960s. In particular, George Dantzig, D. Ray Fulkerson, and Selmer M. Johnson at the RAND Corporation solved the 48-state problem by formulating it as a linear programming problem. The methods they described in their paper on this topic set the foundation for future work in combinatorial optimization, especially highlighting the importance of cutting planes.

In the early 1970s, the concept of P vs. NP problems created excitement in the theoretical computer science community. In 1972, Richard Karp demonstrated that the Hamiltonian cycle problem was NP-complete, implying that the traveling salesman problem was NP-hard.

Increasingly sophisticated codes led to rapid increases in the sizes of the traveling salesman problems solved. Dantzig, Fulkerson, and Johnson had solved a 48-city instance of the problem in 1954. Martin Grötschel more than doubled this 23 years later, solving a 120-city instance in 1977. Harlan Crowder and Manfred W. Padberg again more than doubled this in just 3 years, with a 318-city solution.

In 1987, rapid improvements were made, culminating in a 2,392-city solution by Padberg and Giovanni Rinaldi. In the following two decades, great strides were made with David L. Applegate, Robert E. Bixby, Vasek Chvátal, & William J. Cook solving a 3,308-city instance in 1992, a 7,397-city instance in 1994, a 24,978-city instance in 2004, and an 85,900-city instance in 2006 – which is the largest 2-D Euclidean TSP instance ever solved. William Cook et. al. wrote a program called Concorde TSP Solver for solving the TSP [4]. Concorde is a computer code for the symmetric TSP and some related network optimization problems. The code is written in the ANSI C programming language and it has been used to obtain the optimal solutions to the full set of 110 TSPLIB instances, the largest instance is a 109,399 node 3-D “star” instance.

The continued interest in the TSP can be explained by its success as a general engine of discovery and a steady stream of new applications. Some of the general applications of the TSP are as follows:
* Scheduling and routing problems.
* Genome sequencing.
* Drilling problems.
* Aiming telescopes and x-rays.
* Data clustering.
* Machine scheduling.

We use this classic combinatorial optimization problem to demonstrate how Gurobi can be used to easily and effectively solve small-sized problem instances of the TSP. However, in order to be able to solve larger instances, one needs more sophisticated techniques – such as those implemented in the Concord TSP Solver.

## Problem Description
The TSP can be defined as follows: for a given list of cities and the distances between each pair of them, we want to find the shortest possible route that goes to each city once and returns to the origin city.

There is a class of Traveling Salesman Problems that assumes that the distance of going from city $i$ to city $j$  is the same as going form city $j$ to city $i$, this type of Traveling Salesman Problem  is also known as the symmetric Traveling Salesman Problem. In this example, we use Euclidean distances, but the TSP model formulation is valid independent of the way in which the individual distances are determined.


## Solution Approach

Mathematical programming is a declarative approach where the modeler formulates a mathematical optimization model that captures the key aspects of a complex decision problem. The Gurobi Optimizer solves such models using state-of-the-art mathematics and computer science.

A mathematical optimization model has five components, namely:

* Sets and indices.
* Parameters.
* Decision variables.
* Objective function(s).
* Constraints.

We now present a MIP formulation of the TSP that identifies the shortest route that goes to all the cities once and returns to the origin city.

## TSP Model Formulation

### Sets and Indices
$i, j \in Capitals $: indices and set of US capital cities.

$\text{Pairings}= \{(i,j) \in Capitals \times Capitals \}$: Set of allowed pairings

$S \subset Capitals$: A subset of the set of US capital cities.

$G = (Capitals, Pairings)$: A graph where the set $Capitals$ defines the set of nodes and the set $Pairings$ defines the set of edges.

### Parameters

$d_{i, j} \in \mathbb{R}^+$: Distance from capital city $i$ to capital city $j$, for all $(i, j) \in Pairings$.

Notice that the distance from capital city $i$ to capital city $j$ is the same as the distance from capital city $j$ to capital city $i$, i.e. $d_{i, j} = d_{j, i}$. For this reason, this TSP is also called the symmetric Traveling Salesman Problem.

### Decision Variables
$x_{i, j} \in \{0, 1\}$: This variable is equal to 1, if we decide to connect city $i$ with city $j$. Otherwise, the decision variable is equal to zero.

### Objective Function
- **Shortest Route**. Minimize the total distance of a route. A route is a sequence of capital cities where the salesperson visits each city only once and returns to the starting capital city.

\begin{equation}
\text{Min} \quad Z = \sum_{(i,j) \in \text{Pairings}}d_{i,j} \cdot x_{i,j}
\tag{0}
\end{equation}

### Constraints
- **Symmetry Constraints**. For each edge $(i,j)$, ensure that the city capitals $i$ and $j$ are connected, if the former is visited immediately before or after visiting the latter.

\begin{equation}
x_{i, j} = x_{j, i} \quad \forall (i, j) \in Pairings
\tag{1}
\end{equation}

- **Entering and leaving a capital city**. For each capital city $i$, ensure that this city is connected to two other cities.

\begin{equation}
\sum_{(i,j) \in \text{Pairings}}x_{i,j} = 2 \quad \forall  i \in Capitals
\tag{2}
\end{equation}

- **Subtour elimination**. These constraints ensure that for any subset of cities $S$ of the set of $Capitals$, there is no cycle. That is, there is no route that visits all the cities in the subset and returns to the origin city.

\begin{equation}
\sum_{(i \neq j) \in S}x_{i,j} \leq |S|-1 \quad \forall  S \subset  Capitals
\tag{3}
\end{equation}

- **Remark**. In general, if the number of cities of the TSP is $n$, then the possible number of routes is n\!.
Since there are an exponential number of constraints ($2^{n} - 2$) to eliminate cycles, we use lazy constraints to dynamically eliminate those cycles.

## References

[1] D. L. Applegate, R. E. Bixby, V. Chvatal and W. J. Cook , The Traveling Salesman Problem: A Computational Study, Princeton University Press, Princeton, 2006.

[2] http://www.math.uwaterloo.ca/tsp/index.html

[3] https://www.youtube.com/watch?v=q8nQTNvCrjE&t=35s

[4] http://www.math.uwaterloo.ca/tsp/concorde.html

# Traveling Salesman Problem (TSP) on UK Pubs with Gurobi
This notebook demonstrates how to solve the TSP on a real-world dataset of **[24,727 UK pubs](https://www.math.uwaterloo.ca/tsp/pubs/)**, aiming to find the shortest possible walking tour across the United Kingdom.

We will:
- Configure a **Gurobi WLS (Web License Service) academic license** programmatically
- Load the UK pubs dataset (`uk24727_geom.tsp`)
- Randomly sample 50 / 100 / 200 / 500/ 1000 nodes (we will run 100 as an example)
- Build a Gurobi TSP model with lazy subtour elimination
- Plot the resulting tour on an interactive **Folium** map



## 1) Gurobi License configuration (WLS Academic)
To run large models, you need a valid Gurobi Academic WLS license. Please refers to **[Gurobi License Application Guide](<./Gurobi License Application Guide.md>)** to obtain a `gurobi.lic` file.


**Upload your License:** upload your `gurobi.lic` file to the Colab drive.


![upload-to-colab](images/upload-to-colab.png)

In [55]:
%pip install gurobipy

In [56]:
# ================================================================
# Gurobi License Configuration (WLS Academic License)
# ================================================================
# Replace the placeholders with YOUR credentials (do NOT share keys publicly).
# Students can request free academic licenses here:
# https://www.gurobi.com/academia/academic-program-and-licenses/
# Docs for WLS setup:
# https://support.gurobi.com/hc/en-us/articles/13232844297489-How-do-I-set-up-a-Web-License-Service-WLS-license
# ================================================================

from google.colab import files
import gurobipy as gp

# Step 1: Upload gurobi.lic
print("Please upload gurobi.lic file: ")
uploaded = files.upload()
lic_path = list(uploaded.keys())[0]  # Get the name of the uploaded file（should be `gurobi.lic`）

# Step 2: Parse gurobi.lic
def load_gurobi_params(lic_path):
    params = {}
    with open(lic_path, "r") as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith("#"):  # skip comments
                key, value = line.split("=")
                if key == "LICENSEID":
                    value = int(value)  # LICENSEID must be integer
                params[key] = value
    return params

params = load_gurobi_params(lic_path)

# Step 3: Initialize Gurobi environment
env = gp.Env(params=params)
print("Gurobi version:", gp.gurobi.version())

Please upload gurobi.lic file: 


Saving gurobi.lic to gurobi.lic
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2684114
Academic license 2684114 - for non-commercial use only - registered to y.___@qmul.ac.uk
Gurobi version: (12, 0, 3)


## 2) Imports and helpers

In [57]:
import random
import math
import folium
import numpy as np
from gurobipy import GRB

# For nicer printing
def human(n):
    return f"{n:,}"


## 3) Load UK pubs dataset from TSPLIB-like file

In [58]:
def load_uk_pubs_tsp(file_path: str):
    """Load coordinates from uk24727_geom.tsp keeping the original node ids.
    Returns a dict: {id: (lat, lon)}.
    """
    coords = {}
    with open(file_path, 'r') as f:
        in_section = False
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith("NODE_COORD_SECTION"):
                in_section = True
                continue
            if in_section:
                if line == "EOF":
                    break
                parts = line.split()
                if len(parts) == 3 and parts[0].isdigit():
                    idx, lat, lon = parts
                    coords[int(idx)] = (float(lat), float(lon))
    return coords

# Quick visual sanity-check on a sample (scatter in Folium is done later)
def summary(coords: dict):
    ids = list(coords.keys())
    lats = [coords[i][0] for i in ids]
    lons = [coords[i][1] for i in ids]
    print("Nodes:", len(ids))
    print("Lat range:", (min(lats), max(lats)))
    print("Lon range:", (min(lons), max(lons)))


## 4) Sampling utilities

In [59]:
def sample_nodes(coords: dict, n: int, seed: int = 42) -> dict:
    random.seed(seed)
    chosen = random.sample(list(coords.keys()), n)
    return {k: coords[k] for k in chosen}

def center_of(coords: dict):
    lats = [v[0] for v in coords.values()]
    lons = [v[1] for v in coords.values()]
    return (float(np.mean(lats)), float(np.mean(lons)))

def plot_points_map(coords: dict, title: str = "Sampled UK Pubs"):
    # Compute center
    c_lat, c_lon = center_of(coords)

    # Create map
    fmap = folium.Map(location=[c_lat, c_lon], zoom_start=6, tiles="OpenStreetMap")

    # Add sampled pubs as circle markers
    for pid, (lat, lon) in coords.items():
        folium.CircleMarker(
            location=[lat, lon],
            radius=2,
            popup=f"Pub {pid}",
            color="blue",
            fill=True,
            fill_opacity=0.7
        ).add_to(fmap)

    # Optional: add a layer control if you plan to overlay multiple layers
    folium.LayerControl().add_to(fmap)

    return fmap


## 5) Distance helper

In [60]:
def euclid_dist(i: int, j: int, coords: dict) -> float:
    # Euclidean distance on (lat, lon) as plain coordinates (sufficient for teaching demo).
    (lat_i, lon_i) = coords[i]
    (lat_j, lon_j) = coords[j]
    dx = lat_i - lat_j
    dy = lon_i - lon_j
    return math.hypot(dx, dy)


## 6) Subtour detection (for directed arcs)

In [61]:
def shortest_subtour(selected_arcs, nodes):
    """Return the shortest cycle (list of nodes) from a directed 1-in-1-out solution.
    selected_arcs: iterable of (i, j) pairs with x[i,j] ~ 1
    nodes: list of node ids
    """
    succ = {i: None for i in nodes}
    for i, j in selected_arcs:
        succ[i] = j

    visited = {i: False for i in nodes}
    best_cycle = None

    for start in nodes:
        if visited[start]:
            continue
        cycle = []
        cur = start
        while not visited[cur]:
            visited[cur] = True
            cycle.append(cur)
            cur = succ[cur]
            if cur is None:
                # Shouldn't happen in a feasible solution; break defensively
                break
        # Close the cycle only if it loops back
        if cur in cycle:
            # Trim to the actual cycle
            idx = cycle.index(cur)
            cycle = cycle[idx:]
            if best_cycle is None or len(cycle) < len(best_cycle):
                best_cycle = cycle

    return best_cycle if best_cycle is not None else []


## 7) Build and solve TSP with lazy subtour elimination

In [62]:
def solve_tsp_with_subtour(sample: dict, env: gp.Env, time_limit: float = None):
    nodes = list(sample.keys())
    n = len(nodes)

    # Distance dict for all ordered pairs i != j
    dist = {(i, j): euclid_dist(i, j, sample) for i in nodes for j in nodes if i != j}

    # Build model on the provided environment
    model = gp.Model("tsp_directed", env=env)
    x = model.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name="x")
    # Degree constraints: 1-out and 1-in
    model.addConstrs((gp.quicksum(x[i, j] for j in nodes if j != i) == 1 for i in nodes), name="out")
    model.addConstrs((gp.quicksum(x[j, i] for j in nodes if j != i) == 1 for i in nodes), name="in")

    if time_limit is not None:
        model.Params.TimeLimit = time_limit

    # Store variables for callbacks / extraction
    model._x = x
    model._nodes = nodes
    model.Params.LazyConstraints = 1

    def callback(m, where):
        if where == GRB.Callback.MIPSOL:
            vals = m.cbGetSolution(m._x)  # returns dict keyed by (i,j)
            selected = [(i, j) for (i, j) in m._x.keys() if vals[i, j] > 0.5]
            tour = shortest_subtour(selected, m._nodes)
            if tour and len(tour) < len(m._nodes):
                # Lazy subtour elimination: Sum_{i in S, j in S} x[i,j] <= |S| - 1
                m.cbLazy(gp.quicksum(m._x[i, j] for i in tour for j in tour if i != j) <= len(tour) - 1)

    model.optimize(callback)
    return model


## 8) Plot the optimal tour on a Folium map

In [63]:
def tour_from_solution(model, sample: dict):
    # Use model._x (saved var dict) to avoid 'Model has already been freed' issues
    vals = model.getAttr("x", model._x)
    selected = [(i, j) for (i, j) in model._x.keys() if vals[i, j] > 0.5]
    tour = shortest_subtour(selected, list(sample.keys()))
    assert len(tour) == len(sample), "Subtour found — solution is not a Hamiltonian cycle."
    return tour

def folium_map_for_tour(sample: dict, tour: list):
    c_lat, c_lon = center_of(sample)
    fmap = folium.Map(location=[c_lat, c_lon], zoom_start=6)

    # Build ordered list of points following 'tour'
    points = [(sample[i][0], sample[i][1]) for i in tour]
    points.append(points[0])  # close the loop

    folium.PolyLine(points, weight=3).add_to(fmap)
    for pid, (lat, lon) in sample.items():
        folium.CircleMarker(location=[lat, lon], radius=2, popup=f"Pub {pid}", fill=True).add_to(fmap)
    return fmap


## 9) Run a 100-node example

In [64]:
# Ensure the dataset file is available in the current working directory.
# If running on Colab, upload 'uk24727_geom.tsp' first.
coords_all = load_uk_pubs_tsp('uk24727_geom.tsp')
summary(coords_all)

sample_100 = sample_nodes(coords_all, 100, seed=42)
print("Sample size:", len(sample_100))
plot_points_map(sample_100, title="100 UK Pubs (sample)")

Nodes: 24727
Lat range: (49.968101, 60.560055)
Lon range: (-8.092521, 1.756653)
Sample size: 100


In [65]:
# Solve TSP on 100 sampled pubs
model_100 = solve_tsp_with_subtour(sample_100, env=env)
print("\nStatus:", model_100.Status, "(2=Optimal)")
print("Objective (tour length):", model_100.ObjVal)
print(f"Actual runtime: {model_100.Runtime:.2f} seconds")
print(f"Standardized work units: {model_100.Work:.2f}")

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
LazyConstraints  1

Academic license 2684114 - for non-commercial use only - registered to y.___@qmul.ac.uk
Optimize a model with 200 rows, 9900 columns and 19800 nonzeros
Model fingerprint: 0xa119db15
Variable types: 0 continuous, 9900 integer (9900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-03, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.03s
Presolved: 200 rows, 9900 columns, 19800 nonzeros
Variable types: 0 continuous, 9900 integer (9900 binary)

Root relaxation: objective 2.877816e+01, 164 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |

In [66]:
# Extract and map the tour
tour_100 = tour_from_solution(model_100, sample_100)
fmap_100 = folium_map_for_tour(sample_100, tour_100)
fmap_100

---
### (Optional) Try other sizes
Uncomment and run these to explore scalability.


In [ ]:
# sample_50 = sample_nodes(coords_all, 50, seed=42)
# model_50 = solve_tsp_with_subtour(sample_50, env=env)
# tour_50 = tour_from_solution(model_50, sample_50)
# folium_map_for_tour(sample_50, tour_50)

In [ ]:
# sample_1000 = sample_nodes(coords_all, 1000, seed=42)
# model_1000 = solve_tsp_with_subtour(sample_1000, env=env, time_limit=300)  # optional time limit
# print("Status:", model_1000.Status, "Obj:", model_1000.ObjVal)